In [47]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
You are tasked with generating fusion and inversion functions for arithmetic formulas based on the Semantic Fusion methodology. Your response must strictly adhere to the following steps and output format.
---
**Background:**
Semantic Fusion is a process to combine two formulas \( \phi_1 \) and \( \phi_2 \) into a new fused formula \( \phi_12 \) . This involves:
1. **Fusion Function** \( f(x, y) \): Combines variables \( x \) and \( y \) into a new variable \( z \).
2. **Inversion Functions**:
   - \( r_x(y, z) \): Recovers \( x \) from \( y \) and \( z \).
   - \( r_y(x, z) \): Recovers \( y \) from \( x \) and \( z \).
3. **Fused Formula**: Rewrites the constraints in \( \phi_1 \) and \( \phi_2 \) in terms of \( z \), ensuring that the constraints are equisatisfiable with the originals.


### SMT-LIB Syntax Instructions:
1. Use `define-fun` for defining functions:
   - Example:
     ```smt
     (define-fun f ((x Real) (y Real)) Real
       (+ x y))
     ```
   - This defines \( f(x, y) = x + y \).

2. Use `declare-fun` for variables:
   - Example:
     ```smt
     (declare-fun z () Real)
     ```

3. Use `assert` to define constraints:
   - Example:
     ```smt
     (assert (and (> (- z y) 1) (<= (- z y) 5)))
     ```

4. Avoid invalid constructs such as `f(x y) =` or `verify`. SMT-LIB does not support such constructs.

---

**Output Format:**
Your response must strictly follow this format:

1. **Fusion Function**:
   ```smt
   (define-fun f ((x Real) (y Real)) Real
     [insert fusion function here])

2. **Inversion Functions**:
    (define-fun r_x ((y Real) (z Real)) Real
    [insert inversion function for x])

    (define-fun r_y ((x Real) (z Real)) Real
    [insert inversion function for y])

3. **Fused Formula (SMT-LIB)**:
    (declare-fun z () Real)
    (assert (and [insert fused formula here]))

4.	**Validation**:
    •	Example Inputs: ( x = 2, y = -1 )
    •	Computed ( z = f(x, y) ): [insert result here].
    •	Reconstructed ( x ): [insert result here].
    •	Reconstructed ( y ): [insert result here].
    •	Verify that ( \phi_1 ) and ( \phi_2 ) hold with the reconstructed values.
  
**Notes:**
  1.	Use ( z ) as the fused variable.
  2.	Ensure the fused formula preserves equisatisfiability.
  3.	Provide valid SMT-LIB syntax only.
  4.	Avoid replicating examples; generate outputs specific to the provided input formulas.

        """
        ),
        (
            "human",
            """
**Input Formulas:**
- \( \phi_1: {formula1} \)
- \( \phi_2: {formula2} \)
"""
    ),
]
)

In [48]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="llama3.2",
    temperature=0,
)

In [49]:
chain = prompt | llm
formula1 = "x > 0 ∧ x ≤ 10"
formula2 = "y > 5 ∧ y ≤ 15"

response = chain.invoke({
    "formula1": formula1,
    "formula2": formula2
})

print(response.content)

### Fusion and Inversion Functions for Arithmetic Formulas

#### **Fusion Function**

We define a fusion function that combines variables `x` and `y` into a new variable `z`. The goal is to create a fused formula where both original constraints are preserved.

```smt
(define-fun f ((x Real) (y Real)) Real
  (+ x y))
```

This defines the fusion function as \( z = x + y \).

#### **Inversion Functions**

To recover `x` and `y` from `z`, we define two inversion functions:

- `r_x(y, z)`: Recovers `x` from `y` and `z`.
```smt
(define-fun r_x ((y Real) (z Real)) Real
  (- z y))
```

This recovers `x` as \( x = z - y \).

- `r_y(x, z)`: Recovers `y` from `x` and `z`.
```smt
(define-fun r_y ((x Real) (z Real)) Real
  (/ z x))
```

However, this is not correct. The division operation would result in a non-real value if `x` were zero or negative.

Instead, we can use the following formula to recover `y`:

```smt
(define-fun r_y ((x Real) (z Real)) Real
  (- z (/ z x)))
```

This recovers `y` a